### 라이브러리

In [18]:
import pandas as pd
import os
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from gensim.summarization.summarizer import summarize
from newspaper import Article

### 데이터 전처리

In [ ]:
# # 파일에 있는 모든 csv 파일 불러오기

# # 불러온 파일이 있는 경로
# path = './부산_장애인_교통.csv'
# file_list = os.listdir(path)
# file_list_py = [file for file in file_list if file.endswith('.csv')] # 파일명이 끝이 .csv 인 경우

In [ ]:
# # csv 파일들을 DataFrame으로 불러와서 concat

# news = pd.DataFrame()
# for i in file_list_py:
#     data = pd.read_csv(path+'/'+i, encoding='utf-8')
#     news = pd.concat([news, data])
    
#     news = news.reset_index(drop=True)
# news

In [38]:
news = pd.read_csv('./data/저상버스_불편.csv', encoding='utf-8')
news

,Unnamed: 0.1,Unnamed: 0,title,content
0,0,0,무엇이 ‘장애’를 만들었나[꼬다리],[\r\n\t\t\t[주간경향] 그저 지하철과 버스를 타고 싶을 뿐인데 머리카락을 ...
1,1,1,출근길 시위 ‘매일’ 하겠다…이동권 vs 이동권 충돌 어쩌나,[\r\n [앵커] 장애인 이동권을 보장하라는 출근길 지하철 시위가 22일 만에 재...
2,2,2,남구로·종로3가·청담...휠체어·유아차 가로막는 서울 지하철 21곳,[\r\n엘리베이터 없는 7호선 남구로역종로3가·청담역 등 20곳도 엘리베이터 동선...
3,3,3,"전장연, 지하철 시위 마침표 찍을까…국토부 교통제도 개선 추진","[\r\n국토부, 교통약자법 개정 후속조치…저상버스 도입, 교통비 지원 연구장애인 ..."
4,4,4,"오늘 아침, 출근길은 안녕하셨는지요?",[\r\n\r\n\r\n\r\n\r\n지난달 30일 오전 서울 중구 충무로역에서 휠...
...,...,...,...,...
392,392,392,[뉴있저] 장애인 권리 투쟁으로 우리가 누리는 것들,"[\r\n\t\t\t요즘 장애인 이동권으로 논란이죠, 그런데 장애인 이동권은 '장애..."
393,393,393,"""동등한 이동권 보장을"" 부산 장애인단체, '자립생활' 정책 제안",[\r\n\r\n\r\n\r\n\r\n20일 오후 부산시청 앞에서 부산지역 장애인 ...
394,394,394,"“전장연 사태, 앗 뜨거”… 정치권 불똥 튀자 ‘부랴부랴’ [스토리텔링경제]",[\r\n전장연 ‘이동권 보장’ 시위 논란교통약자법 개정안 내년 시행인데최근 연구 ...
395,395,395,"“길턱에 휘청, 경사에 긴장”…박홍근·고민정 등 민주 ‘휠체어 출근’",[\r\n‘척수장애인’ 최혜영 의원 제안에박홍근 원내대표 등 20여명 참여“턱에 휘...


In [39]:
# news = news.drop(columns=['Unnamed: 0','날짜','기사제목'], axis=0)
# news = news.drop(columns=['Unnamed: 0','날짜'], axis=0)
news = news.drop(columns=['Unnamed: 0','title', 'Unnamed: 0.1'], axis=0)
news["content"] = news["content"].str.replace("\t", "").str.replace(".", "\n")
news

C:\Users\admin\AppData\Local\Temp\ipykernel_12352\1913164047.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  news["content"] = news["content"].str.replace("\t", "").str.replace(".", "\n")


,content
0,[\r\n[주간경향] 그저 지하철과 버스를 타고 싶을 뿐인데 머리카락을 자르는 심정...
1,[\r\n [앵커] 장애인 이동권을 보장하라는 출근길 지하철 시위가 22일 만에 재...
2,[\r\n엘리베이터 없는 7호선 남구로역종로3가·청담역 등 20곳도 엘리베이터 동선...
3,"[\r\n국토부, 교통약자법 개정 후속조치…저상버스 도입, 교통비 지원 연구장애인 ..."
4,[\r\n\r\n\r\n\r\n\r\n지난달 30일 오전 서울 중구 충무로역에서 휠...
...,...
392,"[\r\n요즘 장애인 이동권으로 논란이죠, 그런데 장애인 이동권은 '장애인'만을 위..."
393,[\r\n\r\n\r\n\r\n\r\n20일 오후 부산시청 앞에서 부산지역 장애인 ...
394,[\r\n전장연 ‘이동권 보장’ 시위 논란교통약자법 개정안 내년 시행인데최근 연구 ...
395,[\r\n‘척수장애인’ 최혜영 의원 제안에박홍근 원내대표 등 20여명 참여“턱에 휘...


In [40]:
# 요약 글 데이터프레임으로 만들어주기
for i in range(0, len(news)):
    try:
        text = news['content'][i]
        # 문자열로 만들어주기
        text = (''.join(summarize(text, ratio=0.8).split('\n')))
        # print(text_list)
    except Exception as e:
        print(i,",", end='')
        continue

In [28]:
news = news.drop([92])
news = news.reset_index(drop=True)
news

,기사내용
0,(서울=연합뉴스) 황철환 윤보람 기자 = 유엔이 정한 '제22회 세계 장애인...
1,"[한겨레] 정부, 리프트 설치 비용 내년 예산에 반영 안해장애인단체, 고속버스터미..."
2,[경기] 수원역에서 사당역까지 운행하는 7770번 좌석버스가 수원역 정류장에 당도...
3,【부산=뉴시스】허상천 기자 = 새누리당 권철현 부산시장 예비후보는 10일 “여성 부...
4,[한겨레] 34개 단체 자립권·이동권 투쟁시에 교육·복지·의료지원 요구부산민중연대...
...,...
149,부산시 ‘대중교통 비전 2030’ 2008년 후 계속 40%대 그쳐 하단~녹산선·씨...
150,"국토부, 4차 교통약자 이동편의 증진계획(2022~2026년) 고시내년 1월 시내·..."
151,▲ 경향신문 = 캄보디아 노동자 594편의 '절망기' ▲ 국민일보 = 한·미 ...
152,빈곤층·장애인 잇따라 극단 선택경제적 자립 지원·일자리 창출 실패기초생활수급자 정책...


In [41]:
# 리스트로 담기
sum_list = []

for i in range(0, len(news)):
    text = news['content'][i]
    text_list = (''.join(summarize(text, ratio=0.8).split('\n')))
    
    sum_list.append(text_list)

In [42]:
total = pd.DataFrame(sum_list, columns=['기사요약'])
total

,기사요약
0,[주간경향] 그저 지하철과 버스를 타고 싶을 뿐인데 머리카락을 자르는 심정을 비장애...
1,[앵커] 장애인 이동권을 보장하라는 출근길 지하철 시위가 22일 만에 재개됐습니다단...
2,엘리베이터 없는 7호선 남구로역종로3가·청담역 등 20곳도 엘리베이터 동선 끊겨서울...
3,"국토부, 교통약자법 개정 후속조치…저상버스 도입, 교통비 지원 연구장애인 단체 총 ..."
4,지난달 30일 오전 서울 중구 충무로역에서 휠체어를 탄 전국장애인차별철폐연대 회원이...
...,...
392,"요즘 장애인 이동권으로 논란이죠, 그런데 장애인 이동권은 '장애인'만을 위한 걸까요..."
393,20일 오후 부산시청 앞에서 부산지역 장애인 단체들이 장애인 자립생활 권리보장을 위...
394,전장연 ‘이동권 보장’ 시위 논란교통약자법 개정안 내년 시행인데최근 연구 용역 착수...
395,고민정 더불어민주당 의원이 6일 휠체어를 타고 지하철로 국회에 출근하는 체험을 하고...


In [43]:
total.to_csv('./res/저상버스_불편.csv')